# Detect white references and create a mask covering them.

Import dependencies

In [ ]:
import glob
import cv2
import copy
import numpy as np
import matplotlib.pyplot as plt

Load images

In [ ]:
images = glob.glob("./*/*/*/*[0-9].png")

Find white references and mask them with given margin

In [79]:
save_name = "wref_mask.png"

# Toggle plots on and off
visualize = False

for image in images:
    #Construct save path
    save_path = image.replace(image.split("\\")[-1], "")

    #Load image to memory in grayscale
    cached_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    grayscale_image = copy.deepcopy(cached_image)
    x_dim = cached_image.shape[1]
    y_dim = cached_image.shape[0]

    #Set margin size
    margin_size = 15
    margin_array = []

    #Create margin array
    for p in range(margin_size):
        margin_array.append(False)

    #Sum all pixels over y-axis and reshape array
    sum_over_row = np.sum(cached_image, 1)
    sum_over_row = sum_over_row.reshape(y_dim)

    #Change threshold value based on sensor type
    threshold = 100000 if "FX10" in image else 70000

    #Apply threshold to sum to create binary mask
    vertical_mask = sum_over_row > threshold

    #Add margin to vertical mask
    vertical_mask_padded_top = np.concatenate((vertical_mask[margin_size:], margin_array))
    vertical_mask_padded_bot = np.concatenate((margin_array ,vertical_mask[:len(vertical_mask) - margin_size]))

    #Combine original mask with padding
    combined_mask = vertical_mask | vertical_mask_padded_top | vertical_mask_padded_bot

    #Apply vertical mask to white image to create a binary image
    result = np.ones(cached_image.shape)
    result[combined_mask] = 0

    #Save the binary image
    cv2.imwrite('{}{}'.format(save_path, save_name), result.astype(np.uint8)*255)

    ##-- for visualization purposes only --##

    #Apply mask to grayscale image
    cached_image[combined_mask] = 0

    #Visualize the sum over x-axis and masking process
    if visualize:
        fig, (plt1, plt2) = plt.subplots(2, 1)
        fig.suptitle('sum over y-axis')
        fig.set_figwidth(10)
        plt1.plot(sum_over_row)
        plt2.plot(vertical_mask)

        fig, (plt1, plt2, plt3) = plt.subplots(1, 3)
        fig.suptitle('grayscale image, mask and mask applied to grayscale image')
        plt1.imshow(grayscale_image, cmap="gray")
        plt2.imshow(result, cmap="gray")
        plt3.imshow(cached_image, cmap="gray")